In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sksurv.ensemble import RandomSurvivalForest 
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sklearn.pipeline import  make_pipeline
from sksurv.metrics import concordance_index_censored,concordance_index_ipcw
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import joblib
import seaborn as sns
from scipy.stats import ttest_ind

merge data

In [4]:
train = pd.read_csv('./cli_radi_morphology_3d_DL_nphase_train_id__std.csv')
test = pd.read_csv('./cli_radi_morphology_3d_DL_nphase_test_id__std.csv')
raw_data = pd.read_csv('../202231011給李醫師資料整理/HCC OP+RFA 20230511 n=2261 去名化.csv')
rename_isdnumber = pd.read_csv('../OP_survival/20230523/data/post_early/HCC OP survival data(rename isdnumber).csv')

In [5]:
rename_isdnumber_select = rename_isdnumber[['ISD number', 'OP&RFA Data ID']]
rename_isdnumber_select.rename(columns={'ISD number': 'ISDnumber'}, inplace=True)
rename_isdnumber_select.rename(columns={'OP&RFA Data ID': 'ID'}, inplace=True)
train_id = pd.merge(train, rename_isdnumber_select, on='ISDnumber', how='inner')
test_id = pd.merge(test, rename_isdnumber_select, on='ISDnumber', how='inner')

/var/folders/g6/zb2fhggn4310x47cnly0dj4c0000gn/T/ipykernel_39254/1808311460.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rename_isdnumber_select.rename(columns={'ISD number': 'ISDnumber'}, inplace=True)
/var/folders/g6/zb2fhggn4310x47cnly0dj4c0000gn/T/ipykernel_39254/1808311460.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rename_isdnumber_select.rename(columns={'OP&RFA Data ID': 'ID'}, inplace=True)


In [6]:
train_id.head()

,Tumor size,lnAST,BCLC,Tumor number,ALBIgrade,class_Histologic grade,class_AFP,Ishak,Steatosis grade,K,...,96,97,98,99,dv0,dv1,ISDnumber,Early recur,Early RFS,ID
0,-0.776116,1.046350,-0.272983,1.008003,1.355771,-1.593068,0.434166,1.409544,-0.811318,-1.261013,...,0.344955,-0.882156,-0.485201,0.436020,0.470333,-0.470334,1097.0,1.0,7.13,10741
1,-0.646299,-0.351106,-0.272983,-0.357679,1.355771,-1.593068,-1.093903,1.409544,0.881138,-0.168100,...,-1.713890,0.642984,1.706940,-1.643543,-1.643334,1.643333,497.0,1.0,13.13,10485
2,-0.178961,0.409580,1.276379,1.008003,-0.722418,1.702934,-0.075191,0.384439,-0.811318,0.924812,...,0.269095,-1.461391,-0.377761,0.437109,0.380727,-0.380727,381.0,0.0,24.00,10395
3,1.638466,-0.351106,1.276379,1.008003,-0.722418,0.604267,-0.075191,-1.665771,-0.811318,1.361977,...,-0.287725,0.538154,0.161972,-0.194896,-0.214757,0.214758,1089.0,1.0,12.27,10737
4,0.028745,-0.351106,-0.272983,-0.357679,-0.722418,-1.593068,-0.075191,-0.128114,-0.811318,0.050482,...,-0.828437,1.932147,1.237217,-1.180422,-1.227915,1.227915,350.0,0.0,24.00,10373


In [7]:
raw_data_select = raw_data[['ID', 'Death', 'Overall survival']]
train_overall = pd.merge(train_id, raw_data_select, on='ID', how='inner')
test_overall = pd.merge(test_id, raw_data_select, on='ID', how='inner')

In [8]:
print(len(train),len(train_overall))

410 410


In [9]:
# train_overall.to_csv('./cli_radi_morphology_3d_DL_nphase_train_id__std_overall.csv',index=False)
# test_overall.to_csv('./cli_radi_morphology_3d_DL_nphase_test_id__std_overall.csv',index=False)

model

In [2]:
train = pd.read_csv('./cli_radi_morphology_3d_DL_nphase_train_id__std_overall.csv')
test = pd.read_csv('./cli_radi_morphology_3d_DL_nphase_test_id__std_overall.csv')

In [11]:
selected = ['Tumor size',  'a_Wavelet_rbior_Diagonal_rbio1.5_E0', 'Convex_perimeter-N',
  'a_Wavelet_rbior_Diagonal_rbio3.3_E0','BCLC', 'Tumor number', 'MVI','MaxPixelValue-N', 'n_Wavelet_bior_Vertical_bior3.1_E0',
 'p_Wavelet_bior_Vertical_bior3.1_E0', 'AI_Morphology_score', 'Skewness-N', 'class_Histologic grade', 'lnAST',
   'class_AFP', 'Steatosis grade', 'p_Wavelet_db_Horizontal_4_E0', 'ALBIgrade', 'K', 'AST', 'AI_Morphology_score_classification'
, 'Ishak', 'n_Wavelet_rbior_Vertical_rbio3.3_E0'
,'n_eccentricity','n_compactness','n_euler_pitch','n_counts','n_quaternion_x','n_abb_volume','n_quaternion_w'
,'n_aspect_ratio_3','n_convex_volume',
'28','34','11','7','79','77','36','4','75','n_d2a90_dissimilartiy','n_d4a45_dissimilartiy'
,'p_d4a0_info_measure_corr_2','p_d2a45_correlation','a_d5a135_correlation', 'Hu-1-P','Hu-2-N','Hu-6-A','BMI', 'HBsAg'
]#GARSL2-clinical-radiomic-3D-DL 無 'p_Wavelet_fk_Vertical_fk6_E0'

In [23]:
selected = ['Tumor size',  'a_Wavelet_rbior_Diagonal_rbio1.5_E0', 'Convex_perimeter-N',
  'a_Wavelet_rbior_Diagonal_rbio3.3_E0','BCLC', 'Tumor number', 'MVI','MaxPixelValue-N', 'n_Wavelet_bior_Vertical_bior3.1_E0',
 'p_Wavelet_bior_Vertical_bior3.1_E0', 'AI_Morphology_score', 'Skewness-N', 'class_Histologic grade', 'lnAST',
   'class_AFP', 'Steatosis grade', 'p_Wavelet_db_Horizontal_4_E0', 'ALBIgrade', 'K', 'AST', 'AI_Morphology_score_classification'
, 'p_Wavelet_fk_Vertical_fk6_E0', 'Ishak', 'n_Wavelet_rbior_Vertical_rbio3.3_E0'
,'n_eccentricity','n_compactness','n_euler_pitch','n_counts','n_quaternion_x','n_abb_volume','n_quaternion_w'
,'n_aspect_ratio_3','n_convex_volume',
'28','34','11','7','79','77','36','4','75','BMI', 'HBsAg'
]#GARSL2-clinical-radiomic-3D-DL

In [29]:
X = train[ selected ]
y_struct = train[['Death','Overall survival']].to_records(index=False).astype([('Status', 'bool'), ('Survival', 'float64')])

coxnet = CoxnetSurvivalAnalysis()
coxnet.fit(X, y_struct)
print("fitted.")
preds_train = coxnet.predict(X)
concordance_index_censored(train['Death'] > 0, train['Overall survival'],preds_train)

#concordance_index_ipcw((censored_test, survival_test, preds ))

fitted.


(0.7253000460838727, 36199, 13710, 0, 9)

In [25]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(train['Death'] > 0, preds_train)
print("AUC:", auc)

AUC: 0.7379280492497113


In [30]:
test_X_std = test[train.columns].drop(columns=['Death', 'Overall survival'])
test_X_std = test_X_std[selected]

y_struct_test = test[['Death', 'Overall survival']].to_records(index=False).astype([('Status', 'bool'), ('Survival', 'float64')])
preds = coxnet.predict(test_X_std)
concordance_index_censored(test['Death'] > 0, test['Overall survival'],preds)
#concordance_index_ipcw(y_struct,y_struct_test , preds)

(0.7040087863811093, 6410, 2695, 0, 0)

In [27]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(test['Death'] > 0, preds)
print("AUC:", auc)

AUC: 0.7182718271827182


In [ ]:
selected = ['Tumor size',  'a_Wavelet_rbior_Diagonal_rbio1.5_E0', 'Convex_perimeter-N',
  'a_Wavelet_rbior_Diagonal_rbio3.3_E0','BCLC', 'Tumor number', 'MVI','MaxPixelValue-N', 'n_Wavelet_bior_Vertical_bior3.1_E0',
 'p_Wavelet_bior_Vertical_bior3.1_E0', 'AI_Morphology_score', 'Skewness-N', 'class_Histologic grade', 'lnAST',
   'class_AFP', 'Steatosis grade', 'p_Wavelet_db_Horizontal_4_E0', 'ALBIgrade', 'K', 'AST', 'AI_Morphology_score_classification'
, 'p_Wavelet_fk_Vertical_fk6_E0', 'Ishak', 'n_Wavelet_rbior_Vertical_rbio3.3_E0'
,'n_eccentricity','n_compactness','n_euler_pitch','n_counts','n_quaternion_x','n_abb_volume','n_quaternion_w'
,'n_aspect_ratio_3','n_convex_volume',
'28','34','11','7','79','77','36','4','75','n_d2a90_dissimilartiy','n_d4a45_dissimilartiy'
,'p_d4a0_info_measure_corr_2','p_d2a45_correlation','a_d5a135_correlation', 'Hu-1-P','Hu-2-N','Hu-6-A','BMI', 'HBsAg'
]#GARSL2-clinical-radiomic-3D-DL

In [55]:
max = ['p_Wavelet_bior_Vertical_bior3.1_E0','lnAST','MaxPixelValue-N','AI_Morphology_score_classification','Convex_perimeter-N','p_Wavelet_fk_Vertical_fk6_E0','Tumor number','AST','7','class_AFP','HBsAg','Tumor size','n_Wavelet_bior_Vertical_bior3.1_E0','n_eccentricity','n_aspect_ratio_3','a_Wavelet_rbior_Diagonal_rbio3.3_E0','77','n_abb_volume','Skewness-N','n_euler_pitch','class_Histologic grade','75','36','11','Ishak','n_quaternion_x','BCLC','n_compactness','28','ALBIgrade','n_counts','BMI','p_Wavelet_db_Horizontal_4_E0','n_Wavelet_rbior_Vertical_rbio3.3_E0','Steatosis grade','K','class_Steatosis grade','MVI']
print(len(selected),len(max))

44 38


In [ ]:
ryan = ['n_area_volume_ratio', 'n_surface_area', 'a_d2a0_cluster_prominence', 'n_d1a90_autocorrelation']

In [58]:
import random
# 設定實驗次數
num_experiments = 1000
results_train = []
results_test = []
features = []
required_features = ['Tumor size', 'lnAST' ,'BCLC', 'Tumor number', 'ALBIgrade',
 'class_Histologic grade', 'class_AFP', 'Ishak', 'Steatosis grade', 'K',
 'class_Steatosis grade' ,'AST', 'HBsAg','BMI','MVI',]  # 填入你希望每次都要包含的特徵名稱

for experiment in range(num_experiments):
    # 隨機挑選10個特徵
    random_features = max.copy()
    random.shuffle(random_features)
    random_features = random_features[:len(selected)-10]
    
    # 確保每次都包含某些特徵
    for feat in required_features:
        if feat not in random_features:
            random_features.append(feat)
    
    X_train_random = train[random_features]
    X_test_random = test[random_features]
    # 建立模型
    coxnet = CoxnetSurvivalAnalysis()
    coxnet.fit(X_train_random, y_struct)
    preds_train = coxnet.predict(X_train_random)
    cindex_train = concordance_index_censored(train['Death'] > 0, train['Overall survival'],preds_train)
    preds = coxnet.predict(X_test_random)
    cindex_test = concordance_index_censored(test['Death'] > 0, test['Overall survival'],preds)
    
    if cindex_train[0] > 0.74 and cindex_test[0] > 0.74:
        results_train.append(cindex_train[0])
        results_test.append(cindex_test[0])
        features.append(random_features)
        print(results_train)

print("Done")


Done


44 38


In [60]:
import random
# 設定實驗次數
num_experiments = 10000
results_train = []
results_test = []
features = []

for experiment in range(num_experiments):
    # 隨機挑選10個特徵
    random_features = max.copy()
    random.shuffle(random_features)
    random_features = random_features[:len(selected)-8]

    X_train_random = train[random_features]
    X_test_random = test[random_features]
    # 建立模型
    coxnet = CoxnetSurvivalAnalysis()
    coxnet.fit(X_train_random, y_struct)
    preds_train = coxnet.predict(X_train_random)
    cindex_train = concordance_index_censored(train['Death'] > 0, train['Overall survival'],preds_train)
    preds = coxnet.predict(X_test_random)
    cindex_test = concordance_index_censored(test['Death'] > 0, test['Overall survival'],preds)
    
    if cindex_train[0] >0.74 and cindex_test[0] > 0.74:
        results_train.append(cindex_train[0])
        results_test.append(cindex_test[0])
        features.append(random_features)
        print(results_train)

print("Done")


Done


In [47]:
features

[['n_quaternion_x',
  'n_aspect_ratio_3',
  'class_Histologic grade',
  'p_Wavelet_db_Horizontal_4_E0',
  'HBsAg',
  'a_Wavelet_rbior_Diagonal_rbio3.3_E0',
  'n_Wavelet_rbior_Vertical_rbio3.3_E0',
  'Tumor number',
  '75',
  'Skewness-N',
  'n_convex_volume',
  'Convex_perimeter-N',
  'K',
  '4',
  'AI_Morphology_score',
  'AI_Morphology_score_classification',
  'n_eccentricity',
  'n_compactness',
  'n_Wavelet_bior_Vertical_bior3.1_E0',
  'ALBIgrade',
  '36',
  'p_Wavelet_bior_Vertical_bior3.1_E0',
  'Tumor size',
  'Steatosis grade',
  'MaxPixelValue-N',
  'lnAST',
  'p_Wavelet_fk_Vertical_fk6_E0',
  '77',
  'n_abb_volume',
  'BCLC',
  '11',
  'MVI',
  'Ishak',
  '28',
  'class_AFP',
  'class_Steatosis grade',
  'AST',
  'BMI'],
 ['K',
  'p_Wavelet_fk_Vertical_fk6_E0',
  '11',
  '28',
  'AST',
  'Ishak',
  'lnAST',
  'n_abb_volume',
  'n_counts',
  'a_Wavelet_rbior_Diagonal_rbio1.5_E0',
  'Steatosis grade',
  'p_Wavelet_bior_Vertical_bior3.1_E0',
  'class_Histologic grade',
  'Tumor 

In [46]:
results_test

[0.7009335529928611,
 0.7097199341021416,
 0.7056562328390994,
 0.71334431630972,
 0.7097199341021416,
 0.7032399780340473,
 0.7030203185063152,
 0.7077429983525535,
 0.7069741900054914,
 0.7057660626029654,
 0.7019220208676551,
 0.7063152114222955,
 0.7087314662273476,
 0.7059857221306974,
 0.7085118066996156,
 0.7036792970895113,
 0.7205930807248765,
 0.7044481054365733,
 0.7101592531576058,
 0.714222954420648,
 0.7071938495332235,
 0.7048874244920373,
 0.7047775947281714,
 0.713673805601318,
 0.7055464030752334,
 0.7013728720483251,
 0.7099395936298737,
 0.7127951674903898,
 0.7063152114222955,
 0.7018121911037891,
 0.7103789126853377,
 0.7021416803953872,
 0.7087314662273476,
 0.7069741900054914,
 0.7081823174080176,
 0.700384404173531,
 0.7021416803953872,
 0.7014827018121911,
 0.7091707852828116,
 0.7180669961559583,
 0.7082921471718836,
 0.7045579352004393,
 0.7073036792970895,
 0.7032399780340473,
 0.7064250411861615,
 0.7013728720483251,
 0.7023613399231191,
 0.707633168588687

In [51]:
max = ['p_Wavelet_bior_Vertical_bior3.1_E0','lnAST','MaxPixelValue-N','AI_Morphology_score_classification','Convex_perimeter-N','p_Wavelet_fk_Vertical_fk6_E0','Tumor number','AST','7','class_AFP','HBsAg','Tumor size','n_Wavelet_bior_Vertical_bior3.1_E0','n_eccentricity','n_aspect_ratio_3','a_Wavelet_rbior_Diagonal_rbio3.3_E0','77','n_abb_volume','Skewness-N','n_euler_pitch','class_Histologic grade','75','36','11','Ishak','n_quaternion_x','BCLC','n_compactness','28','ALBIgrade','n_counts','BMI','p_Wavelet_db_Horizontal_4_E0','n_Wavelet_rbior_Vertical_rbio3.3_E0','Steatosis grade','K','class_Steatosis grade','MVI']
print(len(selected),len(max))

44 38


In [5]:
import pandas as pd
from scipy.stats import mannwhitneyu

# Create an empty DataFrame to store results
result_df = pd.DataFrame(columns=['Column', 'p-value'])

# Iterate over each column in the train dataset
for column in train.columns:
    # Calculate p-value using the Mann-Whitney U test
    _, p_value = mannwhitneyu(train[column], train['Death'])
    
    # Append results to the result DataFrame
    result_df = pd.concat([result_df, pd.DataFrame({'Column': [column], 'p-value': [p_value]})], ignore_index=True)

# Sort results by p-value in ascending order
result_df = result_df.sort_values(by='p-value')

# Reset index for continuous indexing
result_df = result_df.reset_index(drop=True)

# Select top 70 columns with smallest p-values
top_70_train_columns = result_df['Column'][:30]

# Print top 70 train columns
print("Top 70 train columns:")
print(top_70_train_columns)

# Write results to a CSV file
top_70_train_columns.to_csv('./pvalue/train_utest_overall.csv', index=False)

# Reset result_df for the test dataset
result_df = pd.DataFrame(columns=['Column', 'p-value'])

# Iterate over each column in the test dataset
for column in test.columns:
    # Calculate p-value using the Mann-Whitney U test
    _, p_value = mannwhitneyu(test[column], test['Death'])
    
    # Append results to the result DataFrame
    result_df = pd.concat([result_df, pd.DataFrame({'Column': [column], 'p-value': [p_value]})], ignore_index=True)

# Sort results by p-value in ascending order
result_df = result_df.sort_values(by='p-value')

# Reset index for continuous indexing
result_df = result_df.reset_index(drop=True)

# Select top 70 columns with smallest p-values
top_70_test_columns = result_df['Column'][:30]

# Print top 70 test columns
print("\nTop 70 test columns:")
print(top_70_test_columns)

# Write results to a CSV file
top_70_test_columns.to_csv('./pvalue/test_utest_overall.csv', index=False)


Top 70 train columns:
0                     Early RFS
1              Overall survival
2                            ID
3                     ISDnumber
4                      n_counts
5                        Hu-2-N
6                  n_abb_volume
7               n_convex_volume
8                        Hu-6-N
9                      n_volume
10                       Hu-1-P
11                 Tumor number
12                       Hu-1-A
13                         BCLC
14               n_surface_area
15    a_d2a0_cluster_prominence
16                n_convex_area
17               n_quaternion_z
18                          AST
19              MaxPixelValue-N
20          n_area_volume_ratio
21      n_d1a90_autocorrelation
22                Compactness-A
23        n_d4a45_dissimilartiy
24               n_quaternion_w
25                  mean_diff-N
26        n_d2a90_dissimilartiy
27              Steatosis grade
28                   Tumor size
29           Convex_perimeter-N
Name: Column, dtyp

In [6]:
train_top_30 = pd.read_csv('./pvalue/train_utest_overall.csv')
test_top_30 = pd.read_csv('./pvalue/test_utest_overall.csv')

# 使用 set.intersection() 找到兩個 CSV 檔案中的交集欄位名稱
common_columns = set(train_top_30['Column']).intersection(test_top_30['Column'])

# 轉換為列表
common_columns_list = list(common_columns)

# 印出共同的欄位名稱
print("共同的欄位名稱:")
print(common_columns_list)

共同的欄位名稱:
['n_area_volume_ratio', 'ISDnumber', 'n_surface_area', 'Overall survival', 'n_d2a90_dissimilartiy', 'Hu-6-N', 'a_d2a0_cluster_prominence', 'n_quaternion_w', 'Tumor size', 'BCLC', 'n_d4a45_dissimilartiy', 'n_d1a90_autocorrelation', 'Hu-1-P', 'n_counts', 'Early RFS', 'n_volume', 'AST', 'n_abb_volume', 'Hu-2-N', 'Tumor number', 'n_convex_area', 'MaxPixelValue-N', 'ID', 'Hu-1-A', 'Compactness-A', 'n_convex_volume']
